In [ ]:
import os

os.chdir("..")
from src.utils import *

print("Current Working Directory ", os.getcwd())

Current Working Directory  c:\Users\filip\Documents\code\SV-eval


In [ ]:
data_dir = "data"
data_type = "noisy"
noise_type = "gaussian"
snr = 0
dataset_name = f"vox1_test_segments_snr_{snr}_noisy_{noise_type}"

noisy_dt_path = os.path.join(data_dir, data_type, noise_type, dataset_name)
if not os.path.exists(noisy_dt_path):
    print(f"Directory {noisy_dt_path} does not exist")
    raise FileNotFoundError(f"Directory {noisy_dt_path} does not exist")
print(f"Directory {noisy_dt_path} exists")

# clean_dataset = scan_directory_voxceleb2(clean_dt_path)

Directory data\noisy\gaussian\vox1_test_segments_snr_0_noisy_gaussian exists


In [8]:
noisy_dataset = scan_directory_voxceleb2(noisy_dt_path)

In [12]:
from df.enhance import enhance, init_df, load_audio, save_audio, resample

model, df_state, _ = init_df()

2025-03-28 11:44:05 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-28 11:44:05 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\filip\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-28 11:44:05 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-28 11:44:05 | INFO     | DF | Found checkpoint C:\Users\filip\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-28 11:44:05 | INFO     | DF | Running on device cuda:0
2025-03-28 11:44:05 | INFO     | DF | Model loaded


c:\Users\filip\Documents\code\SV-eval\.venv\Lib\site-packages\df\checkpoint.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  latest = torch.load(latest, map_location="cp

48000

In [40]:
import warnings
import torchaudio.functional as F


def denoise_dataset_deepfilternet(noisy_dataset, data_dir, subdir, noise_type):
    output_dir = os.path.join(data_dir, subdir, noise_type, dataset_name)

    os.makedirs(output_dir, exist_ok=True)

    enhance_sr = 48000  # for enhancement

    for index, row in tqdm(noisy_dataset.iterrows(), total=noisy_dataset.shape[0]):
        noisy_path = row["path"]

        audio, original_sample_rate = torchaudio.load(noisy_path)
        audio_48k = F.resample(audio, original_sample_rate, enhance_sr)

        enhanced_audio = enhance(model, df_state, audio_48k)
        enhanced_audio_16k = F.resample(
            enhanced_audio, enhance_sr, original_sample_rate
        )

        person_subdir = os.path.join(output_dir, row["person_id"])
        os.makedirs(person_subdir, exist_ok=True)

        enchanced_audio_path = os.path.join(person_subdir, os.path.basename(noisy_path))

        save_audio(enchanced_audio_path, enhanced_audio_16k, sr=original_sample_rate)

In [34]:
noise_types = ["gaussian", "poisson", "rayleigh"]
snrs = [0, 5, 10, 15, 20]
data_dir = "data"
data_type = "noisy"
output_subdir = "deepfilternet3"

for noise_type in noise_types:
    for snr in snrs:
        dataset_name = f"vox1_test_segments_snr_{snr}_noisy_{noise_type}"

        noisy_dt_path = os.path.join(data_dir, data_type, noise_type, dataset_name)
        if not os.path.exists(noisy_dt_path):
            print(f"Directory {noisy_dt_path} does not exist")
            raise FileNotFoundError(f"Directory {noisy_dt_path} does not exist")
        print(f"Directory {noisy_dt_path} exists")

        noisy_dataset = scan_directory_voxceleb2(noisy_dt_path)
        denoise_dataset_deepfilternet(
            noisy_dataset, data_dir, output_subdir, noise_type
        )

Directory data\noisy\gaussian\vox1_test_segments_snr_0_noisy_gaussian exists


100%|██████████| 9119/9119 [21:17<00:00,  7.14it/s]


Directory data\noisy\gaussian\vox1_test_segments_snr_5_noisy_gaussian exists


100%|██████████| 9119/9119 [21:36<00:00,  7.03it/s]


Directory data\noisy\gaussian\vox1_test_segments_snr_10_noisy_gaussian exists


...

100%|██████████| 9119/9119 [20:53<00:00,  7.27it/s]


Directory data\noisy\rayleigh\vox1_test_segments_snr_15_noisy_rayleigh exists


100%|██████████| 9119/9119 [20:25<00:00,  7.44it/s]


Directory data\noisy\rayleigh\vox1_test_segments_snr_20_noisy_rayleigh exists


100%|██████████| 9119/9119 [19:58<00:00,  7.61it/s]


In [41]:
noise_types = ["AirConditioner", "Babble", "Neighbor"]
snrs = [0, 5, 10, 15, 20]
data_dir = "data"
data_type = "noisy_bg\\vox1_test_wav_bq_noise"
output_subdir = "deepfilternet3"

for noise_type in noise_types:
    for snr in snrs:
        dataset_name = f"vox1_test_wav_snr_{snr}_{noise_type}"

        noisy_dt_path = os.path.join(data_dir, data_type, noise_type, dataset_name)
        if not os.path.exists(noisy_dt_path):
            print(f"Directory {noisy_dt_path} does not exist")
            raise FileNotFoundError(f"Directory {noisy_dt_path} does not exist")
        print(f"Directory {noisy_dt_path} exists")

        noisy_dataset = scan_directory_voxceleb2(noisy_dt_path)
        denoise_dataset_deepfilternet(
            noisy_dataset, data_dir, output_subdir, noise_type
        )

Directory data\noisy_bg\vox1_test_wav_bq_noise\AirConditioner\vox1_test_wav_snr_0_AirConditioner exists


100%|██████████| 9119/9119 [20:22<00:00,  7.46it/s]


Directory data\noisy_bg\vox1_test_wav_bq_noise\AirConditioner\vox1_test_wav_snr_5_AirConditioner exists


100%|██████████| 9119/9119 [20:23<00:00,  7.45it/s]


Directory data\noisy_bg\vox1_test_wav_bq_noise\AirConditioner\vox1_test_wav_snr_10_AirConditioner exists


...

100%|██████████| 9119/9119 [20:16<00:00,  7.49it/s]


Directory data\noisy_bg\vox1_test_wav_bq_noise\Neighbor\vox1_test_wav_snr_15_Neighbor exists


100%|██████████| 9119/9119 [20:22<00:00,  7.46it/s]


Directory data\noisy_bg\vox1_test_wav_bq_noise\Neighbor\vox1_test_wav_snr_20_Neighbor exists


100%|██████████| 9119/9119 [20:23<00:00,  7.45it/s]
